In [22]:
from datasets import load_titanic, balance_classes
from tools import model_performance,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_roc_curve
from pygam import LogisticGAM, s, f, l
import numpy as np
np.set_printoptions(suppress=True)

In [23]:
df, df_train, df_test = load_titanic()
df_train.columns

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'target'], dtype='object')

In [24]:
original_X = df.drop(['target'], axis=1)
original_y = df['target']
train_X = df_train.drop(['target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['target'], axis=1)
test_y = df_test['target']

In [25]:
original_X = pd.get_dummies(original_X)
train_X = pd.get_dummies(train_X)
test_X = pd.get_dummies(test_X)

features = train_X.columns

In [26]:
scaler = StandardScaler().fit(train_X)
original_X = scaler.transform(original_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

## Pocet komponent

In [27]:
pca = PCA(n_components=len(features)-1)
pca = pca.fit(original_X)

In [28]:
pca.explained_variance_ratio_

array([0.24580653, 0.20510114, 0.16124175, 0.13768909, 0.08955341,
       0.06604285, 0.05941011, 0.03515511, 0.        ])

## Vizualizace

In [29]:
pca = PCA(n_components=3)
pca = pca.fit(original_X)

In [30]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [31]:
transformed_df = pd.DataFrame(transformed_original_X, columns=['x','y','z'])
transformed_df['target'] = original_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target')

## Classification

In [32]:
pca = PCA(n_components=4)
pca = pca.fit(original_X)

In [33]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [34]:
model = LogisticRegression(penalty='none')

In [35]:
# stratified (balanced), 5-fold - zadne zlepseni oproti ciste LogisticRegression
cross_val_score(model, transformed_original_X, original_y).mean()

0.7811436821291821

In [36]:
model = model.fit(transformed_train_X, train_y)

In [37]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.7808988764044944
Test score
0.7938288920056101


In [38]:
confusion_matrix(model.predict(transformed_train_X), train_y, model.predict(transformed_test_X), test_y)

Train
TN: 93, TP: 46, FN: 22, FP: 17
Test
TN: 379, TP: 187, FN: 87, FP: 60
